Becca Suchower

# Orbital Mechanics Notebook




In [2]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *


Here's a question from the web site [Ask an Astronomer](http://curious.astro.cornell.edu/about-us/39-our-solar-system/the-earth/other-catastrophes/57-how-long-would-it-take-the-earth-to-fall-into-the-sun-intermediate):

"If the Earth suddenly stopped orbiting the Sun, I know eventually it would be pulled in by the Sun's gravity and hit it. How long would it take the Earth to hit the Sun? I imagine it would go slowly at first and then pick up speed."


In [3]:
# Here are the units we'll need

s = UNITS.second
N = UNITS.newton
kg = UNITS.kilogram
m = UNITS.meter

<Unit('meter')>

In [4]:
# And an inition condition (with everything in SI units)

x = 147e9 * m
y = 0 * m
vx = 0 * m/s
vy = 0 * m/s

init = State(x = x,
             y = y,
             vx = vx,
             vy = vy)

,values
x,147000000000.0 meter
y,0 meter
vx,0.0 meter / second
vy,0.0 meter / second


In [5]:
# Making a system object

r_earth = 6.371e6 * m
r_sun = 695.508e6 * m

system = System(init=init,
                G=6.674e-11 * N / kg**2 * m**2,
                m1=1.989e30 * kg,
                r_final=r_sun + r_earth,
                m2=5.972e24 * kg,
                t_0=0 * s,
                t_end=1e7 * s,
                dt=0.1 * s)

,values
init,x 147000000000.0 meter y ...
G,6.674e-11 meter ** 2 * newton / kilogram ** 2
m1,1.989e+30 kilogram
r_final,701879000.0 meter
m2,5.972e+24 kilogram
t_0,0 second
t_end,10000000.0 second
dt,0.1 second


In [8]:
# Here's a function that computes the force of gravity

def universal_gravitation(state, system):
    """Computes gravitational force.
    
    state: State object with distance r
    system: System object with m1, m2, and G
    """
    x, y, vx, vy = state
    unpack(system)
    
    r = Vector(x, y) * m
    
    force = G * m1 * m2 / r.mag**2
    f_vector = -r.hat() * force
    return f_vector

In [9]:
universal_gravitation(init, system)

<Quantity([-3.6686486e+22 -0.0000000e+00], 'newton / meter ** 2')>

In [10]:
# The slope function

def slope_func(state, t, system):
    """Compute derivatives of the state.
    
    state: position, velocity
    t: time
    system: System object containing `g`
    
    returns: derivatives of y and v
    """
    x, y, vx, vy = state
    unpack(system)    
        
    f_vector = universal_gravitation(state, system)
  
    v = Vector(vx, vy)
  
    dvdt = -f_vector / m2
    mag = dvdt.mag
    
    return v.x, v.y

In [22]:
# Always test the slope function!

slope_func(init, 0, system)

(<Quantity(0.0, 'meter / second')>, <Quantity(0.0, 'meter / second')>)

In [ ]:
# Run simulation
results, details = run_ode_solver(system, slope_func, max_step=0.1*s)
details

In [ ]:
#Plot x and y vs time
plot(results.x, label='x')
plot(results.y, label='y')

decorate(xlabel='Time (s)',
         ylabel='Position (m)')

In [ ]:
#Plot trajectory
plot(results.x, results.y, label='trajectory')